In [115]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

glove_file = datapath('/home/nikhil/Desktop/sem8/IP/main/data/glove/glove.6B.300d.txt')
tmp_file = get_tmpfile("word2vec_300.txt")
glove2word2vec(glove_file, tmp_file)
model = KeyedVectors.load_word2vec_format(tmp_file)

In [116]:
import numpy as np
def checkin(a,b):
    for i in range(len(b)):
        if b[i]==a:
            return 1
    return 0

def checkchild(b,word):
    for i in range(len(b)):
        if b[i]==word:
            return 1
    return 0

def checkra(s,shead,b,bhead):
    if (len(s))==1:
        return 0
    #elif bhead[0]==0:
    #    return 0
    else:
        if bhead[0]==s[-1]:
            if checkin(bhead[0],b)==0:
                return 1
    return 0

def checkla(s,shead,b,bhead):
    if (len(s))==1:
        return 0
    #elif shead[-1]==:
    #    return 0
    else:
        if shead[-1]==b[0]:
            if checkchild(bhead,s[-1])==0:
                return 1
    return 0

In [117]:
post=open('../data/vocabs.pos','r').read();
post = post.split('\n')
for i in range(19):
    post[i]=post[i].split(' ')
    post[i]=post[i][0]
dept=open('../data/vocabs.labels','r').read();
dept = dept.split('\n')
for i in range(50):
    dept[i]=dept[i].split(' ')
    dept[i]=dept[i][0]
    
    
def feature(s,b,pos,de,t,post,dept):
    x1=np.array([[0]*300]*1, np.dtype(float))
    try:
        x1=x1+model[s[-1]]
    except:
        pass
    x2=np.array([[0]*300]*1, np.dtype(float))
    try:
        x1=x1+model[b[0]]
    except:
        pass
    x3=np.array([[0]*19]*1, np.dtype(float))
    for i in range(len(t)):
        if s[-1]==t[i]:
            for j in range(19):
                if pos[i]==post[j]:
                    x3[0][j]=1;
    x4=np.array([[0]*19]*1, np.dtype(float))
    for i in range(len(t)):
        if b[0]==t[i]:
            for j in range(19):
                if pos[i]==post[j]:
                    x4[0][j]=1;
    x5=np.array([[0]*50]*1, np.dtype(float))
    for i in range(len(t)):
        if s[-1]==t[i]:
            for j in range(50):
                if dep[i]==dept[j]:
                    x5[0][j]=1;
    x6=np.array([[0]*50]*1, np.dtype(float))
    for i in range(len(t)):
        if b[0]==t[i]:
            for j in range(50):
                if dep[i]==dept[j]:
                    x6[0][j]=1;
    x=np.concatenate((x1,x2,x3,x4,x5,x6),axis=1)
    return x    
    
    
def op_ra(s,sh,b,bh):
    s.pop()
    sh.pop()
    return [s,sh,b,bh]
def op_la(s,sh,b,bh):
    b[0]=s.pop()
    bh[0]=sh.pop()
    return [s,sh,b,bh]
def op_shift(s,sh,b,bh):
    s.append(b[0])
    sh.append(bh[0])
    del(b[0])
    del(bh[0])
    return [s,sh,b,bh]

In [118]:
train=open('train.conllu','r').read();
train = train.split('\n')

In [119]:
t=[]
l=[]
t.append([])
l.append([])
pos=[]
dep=[]
pos.append([])
dep.append([])
j=0;
for i in range(60000):
    try:
       
        if train[i][0] != '#':
            train[i]=train[i].split('\t')
            if(train[i][6]!='_'):
                t[j].append(train[i][1])
                l[j].append(train[i][6])
                pos[j].append(train[i][3])
                dep[j].append(train[i][7])
       #     f.write(train[i]+'\n')
    except:
   #     f.write('\n')
        t.append([])
        l.append([])
        pos.append([])
        dep.append([])
        j=j+1
        pass
#f.close()

In [120]:
X=[]
Y=[]
for i in range(len(l)):
    stack=['#root#']
    stahead=[-1]
    buff=t[i]
    buffhead=[]
    for j in range(len(l[i])):
        if l[i][j]>0:
            #print j
            buffhead.append(t[i][int(l[i][j])-1])
        else:
            buffhead.append('#root#')
    POS=pos[i]
    de=dep[i]
#    print i
#    print buff
#    print buffhead
#    print stack
#    print stahead
    while len(buff)>0:
        if checkra(stack,stahead,buff,buffhead)==1:
            [stack,stahead,buff,buffhead]=op_ra(stack,stahead,buff,buffhead)
            X.append(feature(stack,buff,POS,de,t[i],post,dept))
            Y.append('ra')
        elif checkla(stack,stahead,buff,buffhead)==1:
            [stack,stahead,buff,buffhead]=op_la(stack,stahead,buff,buffhead)
            X.append(feature(stack,buff,POS,de,t[i],post,dept))
            Y.append('la')
        else:
            [stack,stahead,buff,buffhead]=op_shift(stack,stahead,buff,buffhead)
            X.append(feature(stack,buff,POS,de,t[i],post,dept))
            Y.append('shift')
            

In [124]:
print(Y[0])

shift


In [125]:
len(Y)

66146

In [126]:
Yt=np.array([[0]*3]*len(Y), np.dtype(float))
for i in range(len(Y)):
    if(Y[i]=='shift'):
        Yt[i][0]=1
    elif(Y[i]=='ra'):
        Yt[i][1]=1
    else:
        Yt[i][2]=1

In [131]:
np.shape(X)

(66146, 1, 738)

In [132]:
Xtrain=np.array([[0]*738]*50000, np.dtype(float))
for i in range(50000):
    Xtrain[i]=X[i]

In [133]:
np.shape(Xtrain)

(50000, 738)

In [134]:
Xtrain

array([[ 0.040598, -0.028364,  0.25009 , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.040598, -0.028364,  0.25009 , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.040598, -0.028364,  0.25009 , ...,  0.      ,  0.      ,
         0.      ],
       ...,
       [-0.29712 ,  0.094049, -0.096662, ...,  0.      ,  0.      ,
         0.      ],
       [-0.52607 ,  0.059265, -0.223952, ...,  0.      ,  0.      ,
         0.      ],
       [-0.29712 ,  0.094049, -0.096662, ...,  0.      ,  0.      ,
         0.      ]])